In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
from ICARUS.database import Database

# CHANGE THIS TO YOUR DATABASE FOLDER
database_folder = "E:\\Icarus\\Data"

# Load the database
DB = Database(database_folder)
EXTERNAL_DB = DB.EXTERNAL_DB

# Open Database

In [ ]:
DB.vehicles_db.load_all_data()

### Select Plane

In [ ]:
planenames: list[str] = DB.get_vehicle_names()
planenames = [plane for plane in planenames if plane.startswith('hermes')]
for i, planename in enumerate(planenames):
    print(f"{i} : `{planename}")


In [ ]:
from ICARUS.flight_dynamics.state import State

plane = DB.get_vehicle("hermes")
states = DB.get_vehicle_states(plane)
for i, state_name in enumerate(states.keys()):
    print(f"{i} : `{state_name}")
state: State = states['Unstick']

# Add XFLR Polars

In [ ]:
from ICARUS.computation.solvers.XFLR5.polars import read_XFLR5_polars
EXTERNAL_DB = DB.EXTERNAL_DB

for name in planenames:
    try:
        import os
        filename: str = os.path.join(EXTERNAL_DB, f"{name}.txt")
        pol = read_XFLR5_polars(filename, name)
        print(f"Imported XFLR polar for {name}")
    except FileNotFoundError:
        print(f"No XFLR polar found for {name}")
        pass

# Airplane Visualization

In [ ]:
# plane.visualize()

# Polars

In [ ]:
from ICARUS.visualization.airplane.db_polars import plot_airplane_polars

# DB.vehicles_db.load_all_data()
solvers = [
     "GenuVP3 Potential",
    #  "GenuVP3 2D",
    # "GNVP3 ONERA",
    "GenuVP7 Potential",
    # "GenuVP7 2D",
    # "GenuVP7 ONERA",
    #  "LSPT Potential",
    # "LSPT 2D",/'/
    "AVL",
    "XFLR5"
]
axs, fig = plot_airplane_polars(
    # planenames[1:],
    # [planename for planename in planenames if planename.startswith('hermes')],
    ['hermes'],
    # [planename for planename in DB.get_vehicle_names() if planename.startswith('bmark')],
    # 'bmark',
    solvers,
    plots=[["AoA", "CL"], ["AoA", "CD"], ["AoA", "Cm"]],
    size=(6, 7),
)

In [ ]:
fig, axs = state.plot_polars(dimensional=True)

In [ ]:
plane.CG

In [ ]:
state.change_polar_prefix('GenuVP3 Potential')
state.print_trim()

In [ ]:
from ICARUS.environment.definition import EARTH_ISA
import distinctipy

axs, fig = plot_airplane_polars(
    'hermes_high_fidelity',
    solvers,
    plots=[["AoA", "CL"]],
    size=(6, 7),
)


u_inf = 20
rho = EARTH_ISA.air_density
q_inf = 0.5 * rho * u_inf ** 2
visc = EARTH_ISA.air_dynamic_viscosity
sim_reyn = rho * u_inf * plane.main_wing.mean_aerodynamic_chord / visc

axs = axs.flatten()
airfoil = plane.airfoils[0]
airfoil = DB.get_airfoil(airfoil)
polar = DB.get_airfoil_polars(airfoil.name)


correction_factor = 1 + 2 / (plane.aspect_ratio)
print(correction_factor)

# reynolds_nums = polar.reynolds_nums[::4]
reynolds_nums = [polar.reynolds_nums[i] for i in [0, 5, 10 ,15, 20]]

colors = distinctipy.get_colors(len(reynolds_nums))
for i, reyn in enumerate(reynolds_nums):
    df = polar.get_reynolds_subtable(reyn)

    cl = df['CL'] 
    cl3d = cl / correction_factor
    aoa = df['AoA'] - plane.main_wing.orientation[0]

    # Set opacity to 0.5
    axs[0].plot(aoa, cl, label=f'{airfoil.name} {reyn:.2e}', alpha=0.8, linestyle = '--', color=colors[i], linewidth=1)

# Clear all previous legends
fig.legend([], [], loc="lower right", ncol=2)
fig.suptitle(f"{airfoil.name} Cl vs AoA at Re = {sim_reyn:.2e}")
# Get the plot legend
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="lower left", ncol=2)


# reynolds = reynolds_nums[0]
# polar.plot_avl_drag(reynolds)

In [ ]:
reynolds_nums = [polar.reynolds_nums[i] for i in [0, 5, 10 ,15, 20]]

len(polar.reynolds_nums)

In [ ]:
# from ICARUS.visualization.airplane.cg_investigation import cg_investigation

# cg_investigation(
#     planenames[0],
#     solvers,
#     size=(10, 7),
# )

# Case Convergence

In [ ]:
from ICARUS.visualization.airplane.gnvp_convergence import plot_case_convergence
angles2show = [0.0]  # All
# Only Works for unsteady solvers: LSPT, GNVP3, GNVP7
plot_case_convergence(
    plane.name, cases=['Dynamics/m1.875e-01_u'], metrics=["All"], plot_error=False, size=(16, 7)
)

# Wake Visualization

In [ ]:
from ICARUS.visualization.airplane.gnvp_wake import plot_gnvp_wake
from ICARUS.database.utils import angle_to_case
CASE: str = angle_to_case(4.0)
plot_gnvp_wake(3, plane, state, CASE, scale=True)

 # Loads Visualization

In [ ]:
from ICARUS.visualization.airplane.gnvp_strips import gnvp_strips_2d, gnvp_strips_3d
CASE: str = angle_to_case(0.0)

NBs = [i + 1 for i in range(len(plane.surfaces))]
print(plane)
print("NBs to Visualize: ", NBs)

stripData = gnvp_strips_2d(plane, state, CASE, [NBs[0]], gnvp_version= 3, category="Wind")
# stripData = gnvp_strips_3d(plane, state, CASE, [*NBs], gnvp_version= 3, category="FSTRDS2DL(3, IST) / ALSPAN(IST)")

# Dynamics

In [ ]:
from ICARUS.flight_dynamics.state import State

states = DB.get_vehicle_states(plane)

for i, name in enumerate(states):
    print(f"{i}: {name}")

state: State = states["Unstick"]
# state.pertrubation_results

In [ ]:
import jsonpickle


file_path = os.path.join(DB.DB3D, "hermes", "Unstick", "GenuVP3", "Unstick_state.json")
with open(file_path, encoding="UTF-8") as f:
    json_obj: str = f.read()

obj = jsonpickle.decode(json_obj)
if isinstance(obj, State):
    state = obj

In [ ]:
# print(state)
state.stability_fd()
state.plot_eigenvalues()